In [1]:
import pandas as pd
import numpy as np
import datetime

from plotly import graph_objs as go
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

import dowhy

In [2]:
dowhy.__version__

'0.7.1'

In [3]:
import econml
from econml.inference import BootstrapInference
from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
from matplotlib import rcParams
rcParams['font.family'] = 'Times New Roman'
rcParams["font.weight"] = "normal"
rcParams["axes.labelweight"] = "normal"
rcParams.update({'font.size': 15})

In [5]:
%matplotlib inline

In [6]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [19]:
df = pd.read_csv('final_dataset_jan19.csv')

In [20]:
df['Sat'] = df['Sat'].astype('bool')

In [21]:
df['Grav'] = df['Grav'].astype('bool')

# Processing

In [10]:
import sklearn

In [11]:
scaler = sklearn.preprocessing.MinMaxScaler() #  Not yet

In [22]:
df_mid = df.drop(columns=["Head","Grav","Sat","Qr","Beta"])

In [23]:
df_mid.describe()

,Len,BD,Conc,Qs,Ks,Disp,F,Kd,Alpha,q,t,1/R,qe
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,33.750000,1.632325,0.016500,0.322715,0.601239,0.337007,0.356884,0.137761,0.027093,0.142108,646.393386,0.714031,0.002200
std,15.398551,0.076622,0.013092,0.025873,0.465923,0.256402,0.249725,0.127395,0.026524,0.150492,1008.234570,0.264594,0.003309
min,24.000000,1.500000,0.002500,0.264900,0.030000,0.093600,0.082770,0.018700,0.000276,0.020657,29.000000,0.101194,0.000038
25%,24.000000,1.580000,0.008750,0.307850,0.096027,0.155150,0.163200,0.062170,0.010000,0.074075,107.410725,0.526956,0.000130
50%,25.500000,1.615000,0.010000,0.318100,0.726850,0.205850,0.221000,0.085037,0.019855,0.115000,260.714300,0.837182,0.000467
75%,35.250000,1.672500,0.020000,0.339600,1.014000,0.461780,0.508850,0.182500,0.032172,0.166667,516.250000,0.928158,0.003361
max,60.000000,1.800000,0.040000,0.384300,1.391000,1.198000,0.909100,0.643100,0.110220,0.855617,4200.000000,0.990039,0.013395


In [13]:
df_mid.columns 

Index(['Len', 'BD', 'Conc', 'Qs', 'Ks', 'Disp', 'F', 'Kd', 'Alpha', 'q', 't',
       '1/R', 'qe'],
      dtype='object')

In [14]:
df_mid.columns = ['L','rho','C','theta','Ks','Disp','F','Kd','alpha','q','t','R-1','qe']

In [15]:
df_mid.iloc[:,:-3]

,L,rho,C,theta,Ks,Disp,F,Kd,alpha,q
0,27,1.6380,0.0100,0.33000,0.075120,0.15550,0.64550,0.231400,0.008070,0.081487
1,27,1.5362,0.0100,0.32440,0.634200,0.80520,0.22100,0.051070,0.110220,0.855617
2,60,1.7748,0.0100,0.30360,0.099640,1.19800,0.72589,0.384370,0.000276,0.121097
3,27,1.7325,0.0100,0.31460,0.085190,0.17820,0.62620,0.297100,0.027860,0.086580
4,60,1.7377,0.0100,0.31320,0.059592,0.61465,0.44910,0.259400,0.004869,0.056301
5,27,1.7438,0.0100,0.31160,0.051430,0.72117,0.53540,0.376200,0.096910,0.045837
6,27,1.5900,0.0100,0.35630,0.120740,0.17067,0.66160,0.153200,0.083630,0.050930
7,27,1.6300,0.0100,0.34410,0.250170,0.44904,0.47510,0.026560,0.011590,0.051000
8,27,1.7600,0.0100,0.30740,0.065741,0.53063,0.86600,0.125600,0.001494,0.027161
9,27,1.7700,0.0100,0.30480,0.041546,0.21364,0.90910,0.643100,0.003268,0.020657


In [16]:
df_mid.iloc[:,:-3] = scaler.fit_transform(df_mid.iloc[:,:-3])

In [17]:
df_mid

,L,rho,C,theta,Ks,Disp,F,Kd,alpha,q,t,R-1,qe
0,0.083333,0.460000,0.200000,0.545226,0.033152,0.056049,0.680999,0.340647,0.070892,0.072854,380.00000,0.568030,0.001482
1,0.083333,0.120667,0.200000,0.498325,0.443938,0.644332,0.167282,0.051842,1.000000,1.000000,29.00000,0.961849,0.000081
2,1.000000,0.916000,0.200000,0.324121,0.051168,1.000000,0.778285,0.585634,0.000000,0.120293,1030.00000,0.223216,0.005746
3,0.083333,0.775000,0.200000,0.416248,0.040551,0.076603,0.657643,0.445868,0.250892,0.078954,510.00000,0.345029,0.003396
4,1.000000,0.792333,0.200000,0.404523,0.021743,0.471795,0.443322,0.385490,0.041777,0.042690,3900.00000,0.190123,0.007585
5,0.083333,0.812667,0.200000,0.391122,0.015746,0.568245,0.547759,0.572550,0.878938,0.030157,2230.00000,0.150971,0.009940
6,0.083333,0.300000,0.200000,0.765494,0.066672,0.069784,0.700483,0.215407,0.758150,0.036256,342.00000,0.648378,0.001004
7,0.083333,0.433333,0.200000,0.663317,0.161771,0.321840,0.474786,0.012588,0.102908,0.036341,290.00000,0.797787,0.000369
8,0.083333,0.866667,0.200000,0.355946,0.026261,0.395717,0.947842,0.171204,0.011079,0.007789,1450.00000,0.238184,0.004582
9,0.083333,0.900000,0.200000,0.334171,0.008483,0.108693,1.000000,1.000000,0.027215,0.000000,4200.00000,0.101194,0.013395


# Causal DAGs

In [17]:
df_causal_3 = df_mid.drop(columns=['t','qe'])

## Round 4 scaled

In [18]:
#potential obs relationships
causal_graph_4 = """digraph{rho->Disp;rho->Ks;rho->Kd;Disp->"R-1";F->"R-1";alpha->"R-1";L->"R-1";Ks->"R-1";Kd->"R-1";C->"R-1";q->"R-1";L->Disp}"""

In [108]:
params_4 = [('rho','Ks',causal_graph_4),
            ('rho','Kd',causal_graph_4),
            ('rho','Disp',causal_graph_4),
            ('rho','R-1',causal_graph_4),
            ('Disp','R-1',causal_graph_4),
            ('F','R-1',causal_graph_4),
            ('alpha','R-1',causal_graph_4),
            ('L','R-1',causal_graph_4),            
            ('Ks','R-1',causal_graph_4),
            ('Kd','R-1',causal_graph_4),
            ('C','R-1',causal_graph_4),
            ('q','R-1',causal_graph_4),
            ('L','Disp',causal_graph_4)
]

In [42]:
#dml old

'''def causal_process_3(treatment,outcome,graph):
    model = dowhy.CausalModel(data = df_causal_3,
                        treatment = treatment,
                        outcome = outcome,
                        #effect_modifiers = modifiers,
                        graph = graph)
    modifiers = model._effect_modifiers
    if 'U' in modifiers and len(modifiers)>1:
        modifiers.remove('U')
        model = dowhy.CausalModel(data = df_causal_3,
                            treatment = treatment,
                            outcome = outcome,
                            effect_modifiers = modifiers,
                            graph = graph.replace("\n", " "))

    estimand = model.identify_effect(proceed_when_unidentifiable=True,method_name='minimal-adjustment')
    estimate_li = model.estimate_effect(estimand,method_name = "backdoor.linear_regression", method_params = None, confidence_intervals = True)

    res_random_li=model.refute_estimate(estimand,estimate_li, method_name="random_common_cause")
    res_unobserved_li=model.refute_estimate(estimand, estimate_li, method_name="add_unobserved_common_cause",
                                         confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                        effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
    res_placebo_li=model.refute_estimate(estimand, estimate_li, method_name="placebo_treatment_refuter",placebo_type="permute")
    res_sr_li=model.refute_estimate(estimand, estimate_li, method_name="data_subset_refuter",placebo_type="permute")
    li_res = [estimate_li.value, res_random_li,res_unobserved_li,res_placebo_li,res_sr_li, estimate_li.get_confidence_intervals()]

    if 'U' not in modifiers and len(modifiers)>0:
        est_nonparam = CausalForestDML(model_y=GradientBoostingRegressor(), model_t=GradientBoostingRegressor(),random_state=12,inference=True)

        X = df_causal_3[modifiers].values
        Y = df_causal_3[outcome].values
        T = df_causal_3[treatment].values
        est_nonparam_dw = est_nonparam.dowhy.fit(Y,T,X=X,outcome_names=[outcome], treatment_names=[treatment],
                                                feature_names= modifiers, graph = graph,inference = 'auto')
        te_pred = est_nonparam_dw.effect(X)
        estimated_ate = te_pred.mean()
        estimate_ate_2 = est_nonparam_dw.ate(X)
        estimated_ate_ci = est_nonparam_dw.ate_interval(X)

        res_random_dml = est_nonparam_dw.refute_estimate(method_name="random_common_cause")
        res_unobserved_dml = est_nonparam_dw.refute_estimate(method_name="add_unobserved_common_cause",
                                             confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                            effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
        res_placebo_dml = est_nonparam_dw.refute_estimate(method_name="placebo_treatment_refuter", placebo_type="permute", num_simulations=100)
        res_sr_dml = est_nonparam_dw.refute_estimate(method_name="data_subset_refuter")

        dml_res = [estimated_ate, res_random_dml,res_unobserved_dml,res_placebo_dml,res_sr_dml,estimate_ate_2,estimated_ate_ci]
    else:
        dml_res = None

    return li_res,dml_res,modifiers'''

In [114]:
def causal_process_3(treatment,outcome,graph):
    model = dowhy.CausalModel(data = df_causal_3,
                        treatment = treatment,
                        outcome = outcome,
                        #effect_modifiers = modifiers,
                        graph = graph)
    modifiers = model._effect_modifiers
    if 'U' in modifiers and len(modifiers)>1:
        modifiers.remove('U')
        model = dowhy.CausalModel(data = df_causal_3,
                            treatment = treatment,
                            outcome = outcome,
                            effect_modifiers = modifiers,
                            graph = graph.replace("\n", " "))


    estimand = model.identify_effect(proceed_when_unidentifiable=True)
    
    backdoor_var = estimand.get_backdoor_variables()
    
    estimate_li = model.estimate_effect(estimand,method_name = "backdoor.linear_regression", method_params = None, confidence_intervals = True)
    res_random_li=model.refute_estimate(estimand,estimate_li, method_name="random_common_cause")
    res_unobserved_li=model.refute_estimate(estimand, estimate_li, method_name="add_unobserved_common_cause",
                                         confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                         effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
    res_placebo_li=model.refute_estimate(estimand, estimate_li, method_name="placebo_treatment_refuter",placebo_type="permute")
    res_sr_li=model.refute_estimate(estimand, estimate_li, method_name="data_subset_refuter",placebo_type="permute")
    li_ci = estimate_li.get_confidence_intervals()
    
    li_res = [estimate_li.value, res_random_li,res_unobserved_li,res_placebo_li,res_sr_li, li_ci]
    

    dml_estimate = model.estimate_effect(estimand, method_name="backdoor.econml.dml.CausalForestDML",
                                 target_units = "ate",
                                 confidence_intervals=True,
                                 method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                              'model_t': GradientBoostingRegressor()},
                                               "fit_params":{}})
        
    res_random_dml=model.refute_estimate(estimand,dml_estimate, method_name="random_common_cause")
    res_unobserved_dml=model.refute_estimate(estimand, dml_estimate, method_name="add_unobserved_common_cause",
                                             confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                             effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
    res_placebo_dml=model.refute_estimate(estimand, dml_estimate, method_name="placebo_treatment_refuter",placebo_type="permute")
    res_sr_dml=model.refute_estimate(estimand, dml_estimate, method_name="data_subset_refuter",placebo_type="permute")
    dml_ci = dml_estimate.get_confidence_intervals()
        
    dml_res = [dml_estimate.value, res_random_dml,res_unobserved_dml,res_placebo_dml,res_sr_dml,dml_ci]

        
    return li_res,dml_res,backdoor_var,modifiers

In [78]:
def causal_process_ind(treatment,outcome,graph):
    model = dowhy.CausalModel(data = df_causal_3,
                        treatment = treatment,
                        outcome = outcome,
                        #effect_modifiers = modifiers,
                        graph = graph)
    modifiers = model._effect_modifiers
    if 'U' in modifiers and len(modifiers)>1:
        modifiers.remove('U')
        model = dowhy.CausalModel(data = df_causal_3,
                            treatment = treatment,
                            outcome = outcome,
                            effect_modifiers = modifiers,
                            graph = graph.replace("\n", " "))

    estimand = model.identify_effect(proceed_when_unidentifiable=True)
    estimate_li = model.estimate_effect(estimand,method_name = "backdoor.linear_regression", method_params = None, confidence_intervals = True)

    res_random_li=model.refute_estimate(estimand,estimate_li, method_name="random_common_cause")
    res_unobserved_li=model.refute_estimate(estimand, estimate_li, method_name="add_unobserved_common_cause",
                                         confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                        effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
    res_placebo_li=model.refute_estimate(estimand, estimate_li, method_name="placebo_treatment_refuter",placebo_type="permute")
    res_sr_li=None
    li_res = [estimate_li.value, res_random_li,res_unobserved_li,res_placebo_li,res_sr_li, estimate_li.get_confidence_intervals()]

    if 'U' not in modifiers and len(modifiers)>0:
        est_nonparam = CausalForestDML(model_y=GradientBoostingRegressor(), model_t=GradientBoostingRegressor(),random_state=12)

        X = df_causal_3[modifiers].values
        Y = df_causal_3[outcome].values
        T = df_causal_3[treatment].values
        est_nonparam_dw = est_nonparam.dowhy.fit(Y,T,X=X,outcome_names=[outcome], treatment_names=[treatment],
                                                feature_names= modifiers, graph = graph,inference = 'auto')
        te_pred = est_nonparam_dw.effect(X)
        estimated_ate = te_pred.mean()
        estimate_ate_2 = est_nonparam_dw.ate(X)
        estimated_ate_ci = est_nonparam_dw.ate_interval(X)

        res_random_dml = est_nonparam_dw.refute_estimate(method_name="random_common_cause")
        res_unobserved_dml = est_nonparam_dw.refute_estimate(method_name="add_unobserved_common_cause",
                                             confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                            effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
        res_placebo_dml = est_nonparam_dw.refute_estimate(method_name="placebo_treatment_refuter", placebo_type="permute", num_simulations=100)
        res_sr_dml = est_nonparam_dw.refute_estimate(method_name="data_subset_refuter")

        dml_res = [estimated_ate, res_random_dml,res_unobserved_dml,res_placebo_dml,res_sr_dml,estimate_ate_2,estimated_ate_ci]
    else:
        dml_res = None

    return li_res,dml_res,modifiers,estimand

In [110]:
params_4.pop(2)

('rho',
 'Disp',
 'digraph{rho->Disp;rho->Ks;rho->Kd;Disp->"R-1";F->"R-1";alpha->"R-1";L->"R-1";Ks->"R-1";Kd->"R-1";C->"R-1";q->"R-1";L->Disp}')

In [115]:
len(params_4)

In [116]:
result_causal_4=[]
for i in params_4:
        result_causal_4.append(causal_process_3(*i))

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': Fals

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effe

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effe

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effe

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effe

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effe

ValueError: This estimator does not support X=None!

In [48]:
df_res_4=pd.DataFrame(columns = ['link','ate_li','rand_li','unobserved_li','placebo_li','subset_li','li-pl-p-val','li_sr_p_val','ate_dml','rand_dml','unobserved_dml','placebo_dml','subset_dml','dml_pl_p_val','dml_sr_p_val','modifiers','ci_li','ci_dml','ate_dml_2'])

In [49]:
df_res_4['link'] = params_4

In [50]:
df_res_4['ate_li'] = [x[0][0] for x in result_causal_4]
df_res_4['rand_li'] = [x[0][1].new_effect for x in result_causal_4]
df_res_4['unobserved_li'] = [x[0][2].new_effect for x in result_causal_4]
df_res_4['placebo_li'] = [x[0][3].new_effect for x in result_causal_4]
df_res_4['subset_li'] = [x[0][4].new_effect for x in result_causal_4]
df_res_4['li-pl-p-val'] = [x[0][3].refutation_result['p_value'] for x in result_causal_4]
df_res_4['li_sr_p_val'] = [x[0][4].refutation_result['p_value'] for x in result_causal_4]

In [51]:
df_res_4['ate_dml'] = [x[1][0] if x[1] else None for x in result_causal_4]
df_res_4['rand_dml'] = [x[1][1].new_effect if x[1] else None for x in result_causal_4]
df_res_4['unobserved_dml'] = [x[1][2].new_effect if x[1] else None for x in result_causal_4]
df_res_4['placebo_dml'] = [x[1][3].new_effect if x[1] else None for x in result_causal_4]
df_res_4['subset_dml'] = [x[1][4].new_effect if x[1] else None for x in result_causal_4]
df_res_4['dml_pl_p_val'] = [x[1][3].refutation_result['p_value'] if x[1] else None for x in result_causal_4]
df_res_4['dml_sr_p_val'] = [x[1][4].refutation_result['p_value'] if x[1] else None for x in result_causal_4]

In [54]:
result_causal_4

[([-0.32999984236801394,
   array([[-0.55113669, -0.10886299]])],
  None,
  []),
 ([0.45375520670889324,
   array([[0.22746037, 0.68005004]])],
  None,
  []),
 ([-0.3247044201569892,
   array([[-0.73450522,  0.36504672]])],
  [-0.38330361728400875,
   -0.38330361728400864,
   (-0.6652566289063622, -0.10135060566165521)],
  ['q', 'L', 'C', 'alpha', 'F']),
 ([0.05271813501951439,
   array([[-2.10669692,  0.17589951]])],
  [-0.08227559306695671,
   -0.08227559306695671,
   (-0.2727186912054259, 0.10816750507151246)],
  ['Kd', 'q', 'C', 'alpha', 'Ks', 'F']),
 ([-0.0646059671532937,
   array([[-0.95208855,  0.66112895]])],
  [-0.44373839377121466,
   -0.4437383937712144,
   (-0.7546950831810653, -0.13278170436136394)],
  ['Kd', 'q', 'Disp', 'alpha', 'Ks', 'L', 'C', 'rho']),
 ([0.2862241055543093,
   array([[-1.20728587,  3.05894703]])],
  [-0.12978650160550084,
   -0.12978650160550081,
   (-0.30138845422338045, 0.04181545101237874)],
  ['Kd', 'q', 'Disp', 'Ks', 'L', 'C', 'rho', 'F']),
 ([0.

In [55]:
df_res_4['ate_dml_2'] = [x[1][5] if x[1] else None for x in result_causal_4]
df_res_4['ci_li'] = [x[0][5] for x in result_causal_4]

df_res_4['ci_dml'] = [x[1][6] if x[1] else None for x in result_causal_4]

In [56]:
df_res_4['modifiers'] = [x[2] for x in result_causal_4]

In [57]:
df_res_4

,link,ate_li,rand_li,unobserved_li,placebo_li,subset_li,li-pl-p-val,li_sr_p_val,ate_dml,rand_dml,unobserved_dml,placebo_dml,subset_dml,dml_pl_p_val,dml_sr_p_val,modifiers,ci_li,ci_dml,ate_dml_2
0,"(rho, Ks, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",-0.330000,-0.342924,-0.3502797828529346,0.012441,-0.333824,0.46,0.44,NaN,NaN,None,NaN,NaN,NaN,NaN,[],"[[-0.55113669232979, -0.10886299240623809]]",None,NaN
1,"(rho, Kd, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",0.453755,0.457419,0.4537972674157295,0.017344,0.433337,0.42,0.43,NaN,NaN,None,NaN,NaN,NaN,NaN,[],"[[0.22746036993565955, 0.6800500434821269]]",None,NaN
2,"(rho, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;D...",-0.324704,-0.325119,-0.32750049094016853,-0.001246,-0.324287,0.49,0.49,-0.383304,-0.222708,-0.31427078599011354,0.010583,-0.492619,0.44,0.20,"[q, L, C, alpha, F]","[[-0.7345052242201049, 0.36504671785921355]]","(-0.6652566289063622, -0.10135060566165521)",-0.383304
3,"(Disp, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;...",0.052718,0.056040,0.060054542684765067,0.022577,0.010540,0.36,0.40,-0.082276,-0.087303,-0.08288438175819493,-0.006105,-0.121831,0.47,0.40,"[Kd, q, C, alpha, Ks, F]","[[-2.1066969240308335, 0.17589950671180166]]","(-0.2727186912054259, 0.10816750507151246)",-0.082276
4,"(F, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",-0.064606,-0.064376,-0.03976211584463063,0.007524,-0.024201,0.42,0.40,-0.443738,-0.362357,-0.3434001564365298,-0.013230,-0.300089,0.39,0.11,"[Kd, q, Disp, alpha, Ks, L, C, rho]","[[-0.952088545247299, 0.6611289500100209]]","(-0.7546950831810653, -0.13278170436136394)",-0.443738
5,"(alpha, R-1, digraph{rho->Disp;rho->Ks;rho->Kd...",0.286224,0.272661,0.2779504548287367,0.001215,0.396547,0.49,0.47,-0.129787,-0.121600,-0.14369563943836577,-0.003750,0.080509,0.46,0.03,"[Kd, q, Disp, Ks, L, C, rho, F]","[[-1.2072858736587033, 3.0589470277178368]]","(-0.30138845422338045, 0.04181545101237874)",-0.129787
6,"(L, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",0.144271,0.123504,0.1254021381092274,0.018447,-0.184758,0.41,0.31,0.023481,0.022258,0.02656155852514065,-0.004871,-0.001672,0.44,0.31,"[Kd, rho, q, C, alpha, Ks, F]","[[-0.9347825907011025, 0.47828210345634764]]","(-0.0390175611772469, 0.08597956856729516)",0.023481
7,"(Ks, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",0.365239,0.350607,0.3936025577460116,-0.008622,0.386989,0.37,0.38,0.248724,0.435125,0.2565266715983058,0.004556,0.143525,0.46,0.28,"[Kd, F, q, Disp, C, alpha, L]","[[-0.5261415436673003, 0.5804747236052208]]","(-0.020736098363237032, 0.5181845809595654)",0.248724
8,"(Kd, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",-0.738747,-0.731954,-0.6111834694960909,0.074398,-0.501248,0.34,0.30,-0.376761,-0.495855,-0.45420774975363354,-0.021335,-0.603071,0.38,0.18,"[F, q, Disp, C, alpha, Ks, L]","[[-3.905180055706234, -0.6562753667381214]]","(-0.6178182398930463, -0.135703814241236)",-0.376761
9,"(C, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",0.096289,0.099355,0.07422439759409916,0.005475,0.103954,0.40,0.48,0.090944,0.081929,0.06322807723405906,-0.002936,0.090625,0.43,0.47,"[Kd, q, Disp, alpha, Ks, L, rho, F]","[[-1.083784170431142, 1.339096203280483]]","(0.026587254889049228, 0.15530142069676267)",0.090944


In [58]:
df_res_4.to_csv("causal_result_scaled_ci.csv")

In [79]:
res_ind = causal_process_ind('rho',
 'Disp',
 causal_graph_4)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please cha

In [86]:
res_ind[-1].backdoor_variables

{'backdoor1': [],
 'backdoor2': ['q', 'alpha', 'L', 'C', 'F'],
 'backdoor': ['q', 'alpha', 'L', 'C', 'F']}

In [84]:
res_ind

([0.4447779829347664,
  None,
  array([[0.07956198, 0.78667017]])],
 [0.08347525215882921,
  0.08347525215882917,
  (-0.3202412723371771, 0.48719177665483543)],
 ['L'],
 <dowhy.causal_identifier.IdentifiedEstimand at 0x24b60541c08>)

In [62]:
df_res_ind=pd.DataFrame(columns = ['link','ate_li','rand_li','unobserved_li','placebo_li','subset_li','li-pl-p-val','li_sr_p_val','ate_dml','rand_dml','unobserved_dml','placebo_dml','subset_dml','dml_pl_p_val','dml_sr_p_val','modifiers','ci_li','ci_dml','ate_dml_2'])

In [63]:
df_res_ind['modifiers'] = [res_ind[2]]

In [64]:
df_res_ind.link = [('rho','Disp')]

In [65]:
df_res_ind['ate_li'] = [res_ind[0][0]]
df_res_ind['rand_li'] = [res_ind[0][1].new_effect]
df_res_ind['unobserved_li'] = [res_ind[0][2].new_effect]
df_res_ind['placebo_li'] =  [res_ind[0][3].new_effect]
df_res_ind['subset_li'] =  [None]
df_res_ind['li-pl-p-val'] = [res_ind[0][3].refutation_result['p_value']]
df_res_ind['li_sr_p_val'] = [None]

In [66]:
df_res_ind['ate_dml'] = [res_ind[1][0]]
df_res_ind['rand_dml'] = [res_ind[1][1].new_effect]
df_res_ind['unobserved_dml'] = [res_ind[1][2].new_effect]
df_res_ind['placebo_dml'] = [res_ind[1][3].new_effect]
df_res_ind['subset_dml'] = [None]
df_res_ind['dml_pl_p_val'] = [res_ind[1][3].refutation_result['p_value']]
df_res_ind['dml_sr_p_val'] = [None]

In [67]:
df_res_ind['ate_dml_2'] = [res_ind[1][5]]
df_res_ind['ci_li'] = [res_ind[0][5]]

df_res_ind['ci_dml'] = [res_ind[1][6]]

In [68]:
df_res_ind

,link,ate_li,rand_li,unobserved_li,placebo_li,subset_li,li-pl-p-val,li_sr_p_val,ate_dml,rand_dml,unobserved_dml,placebo_dml,subset_dml,dml_pl_p_val,dml_sr_p_val,modifiers,ci_li,ci_dml,ate_dml_2
0,"(rho, Disp)",0.444778,0.444332,0.44973277533937767,0.018541,None,0.44,None,0.083475,0.281098,0.054270741416893684,-0.007002,None,0.5,None,[L],"[[0.07956197545648686, 0.7866701734030466]]","(-0.3202412723371772, 0.48719177665483554)",0.083475


In [69]:
df_res_4.

,link,ate_li,rand_li,unobserved_li,placebo_li,subset_li,li-pl-p-val,li_sr_p_val,ate_dml,rand_dml,unobserved_dml,placebo_dml,subset_dml,dml_pl_p_val,dml_sr_p_val,modifiers,ci_li,ci_dml,ate_dml_2
0,"(rho, Ks, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",-0.330000,-0.342924,-0.3502797828529346,0.012441,-0.333824,0.46,0.44,NaN,NaN,None,NaN,NaN,NaN,NaN,[],"[[-0.55113669232979, -0.10886299240623809]]",None,NaN
1,"(rho, Kd, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",0.453755,0.457419,0.4537972674157295,0.017344,0.433337,0.42,0.43,NaN,NaN,None,NaN,NaN,NaN,NaN,[],"[[0.22746036993565955, 0.6800500434821269]]",None,NaN
2,"(rho, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;D...",-0.324704,-0.325119,-0.32750049094016853,-0.001246,-0.324287,0.49,0.49,-0.383304,-0.222708,-0.31427078599011354,0.010583,-0.492619,0.44,0.20,"[q, L, C, alpha, F]","[[-0.7345052242201049, 0.36504671785921355]]","(-0.6652566289063622, -0.10135060566165521)",-0.383304
3,"(Disp, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;...",0.052718,0.056040,0.060054542684765067,0.022577,0.010540,0.36,0.40,-0.082276,-0.087303,-0.08288438175819493,-0.006105,-0.121831,0.47,0.40,"[Kd, q, C, alpha, Ks, F]","[[-2.1066969240308335, 0.17589950671180166]]","(-0.2727186912054259, 0.10816750507151246)",-0.082276
4,"(F, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",-0.064606,-0.064376,-0.03976211584463063,0.007524,-0.024201,0.42,0.40,-0.443738,-0.362357,-0.3434001564365298,-0.013230,-0.300089,0.39,0.11,"[Kd, q, Disp, alpha, Ks, L, C, rho]","[[-0.952088545247299, 0.6611289500100209]]","(-0.7546950831810653, -0.13278170436136394)",-0.443738
5,"(alpha, R-1, digraph{rho->Disp;rho->Ks;rho->Kd...",0.286224,0.272661,0.2779504548287367,0.001215,0.396547,0.49,0.47,-0.129787,-0.121600,-0.14369563943836577,-0.003750,0.080509,0.46,0.03,"[Kd, q, Disp, Ks, L, C, rho, F]","[[-1.2072858736587033, 3.0589470277178368]]","(-0.30138845422338045, 0.04181545101237874)",-0.129787
6,"(L, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",0.144271,0.123504,0.1254021381092274,0.018447,-0.184758,0.41,0.31,0.023481,0.022258,0.02656155852514065,-0.004871,-0.001672,0.44,0.31,"[Kd, rho, q, C, alpha, Ks, F]","[[-0.9347825907011025, 0.47828210345634764]]","(-0.0390175611772469, 0.08597956856729516)",0.023481
7,"(Ks, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",0.365239,0.350607,0.3936025577460116,-0.008622,0.386989,0.37,0.38,0.248724,0.435125,0.2565266715983058,0.004556,0.143525,0.46,0.28,"[Kd, F, q, Disp, C, alpha, L]","[[-0.5261415436673003, 0.5804747236052208]]","(-0.020736098363237032, 0.5181845809595654)",0.248724
8,"(Kd, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",-0.738747,-0.731954,-0.6111834694960909,0.074398,-0.501248,0.34,0.30,-0.376761,-0.495855,-0.45420774975363354,-0.021335,-0.603071,0.38,0.18,"[F, q, Disp, C, alpha, Ks, L]","[[-3.905180055706234, -0.6562753667381214]]","(-0.6178182398930463, -0.135703814241236)",-0.376761
9,"(C, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",0.096289,0.099355,0.07422439759409916,0.005475,0.103954,0.40,0.48,0.090944,0.081929,0.06322807723405906,-0.002936,0.090625,0.43,0.47,"[Kd, q, Disp, alpha, Ks, L, rho, F]","[[-1.083784170431142, 1.339096203280483]]","(0.026587254889049228, 0.15530142069676267)",0.090944


In [74]:
df_merged = pd.concat([df_res_4,df_res_ind]).reset_index()

In [76]:
df_merged.to_csv("causal_result_scaled_ci.csv")

In [77]:
df_merged

,index,link,ate_li,rand_li,unobserved_li,placebo_li,subset_li,li-pl-p-val,li_sr_p_val,ate_dml,rand_dml,unobserved_dml,placebo_dml,subset_dml,dml_pl_p_val,dml_sr_p_val,modifiers,ci_li,ci_dml,ate_dml_2
0,0,"(rho, Ks, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",-0.330000,-0.342924,-0.3502797828529346,0.012441,-0.333824,0.46,0.44,NaN,NaN,None,NaN,NaN,NaN,NaN,[],"[[-0.55113669232979, -0.10886299240623809]]",None,NaN
1,1,"(rho, Kd, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",0.453755,0.457419,0.4537972674157295,0.017344,0.433337,0.42,0.43,NaN,NaN,None,NaN,NaN,NaN,NaN,[],"[[0.22746036993565955, 0.6800500434821269]]",None,NaN
2,2,"(rho, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;D...",-0.324704,-0.325119,-0.32750049094016853,-0.001246,-0.324287,0.49,0.49,-0.383304,-0.222708,-0.31427078599011354,0.010583,-0.492619,0.44,0.2,"[q, L, C, alpha, F]","[[-0.7345052242201049, 0.36504671785921355]]","(-0.6652566289063622, -0.10135060566165521)",-0.383304
3,3,"(Disp, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;...",0.052718,0.056040,0.060054542684765067,0.022577,0.01054,0.36,0.4,-0.082276,-0.087303,-0.08288438175819493,-0.006105,-0.121831,0.47,0.4,"[Kd, q, C, alpha, Ks, F]","[[-2.1066969240308335, 0.17589950671180166]]","(-0.2727186912054259, 0.10816750507151246)",-0.082276
4,4,"(F, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",-0.064606,-0.064376,-0.03976211584463063,0.007524,-0.024201,0.42,0.4,-0.443738,-0.362357,-0.3434001564365298,-0.013230,-0.300089,0.39,0.11,"[Kd, q, Disp, alpha, Ks, L, C, rho]","[[-0.952088545247299, 0.6611289500100209]]","(-0.7546950831810653, -0.13278170436136394)",-0.443738
5,5,"(alpha, R-1, digraph{rho->Disp;rho->Ks;rho->Kd...",0.286224,0.272661,0.2779504548287367,0.001215,0.396547,0.49,0.47,-0.129787,-0.121600,-0.14369563943836577,-0.003750,0.080509,0.46,0.03,"[Kd, q, Disp, Ks, L, C, rho, F]","[[-1.2072858736587033, 3.0589470277178368]]","(-0.30138845422338045, 0.04181545101237874)",-0.129787
6,6,"(L, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",0.144271,0.123504,0.1254021381092274,0.018447,-0.184758,0.41,0.31,0.023481,0.022258,0.02656155852514065,-0.004871,-0.001672,0.44,0.31,"[Kd, rho, q, C, alpha, Ks, F]","[[-0.9347825907011025, 0.47828210345634764]]","(-0.0390175611772469, 0.08597956856729516)",0.023481
7,7,"(Ks, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",0.365239,0.350607,0.3936025577460116,-0.008622,0.386989,0.37,0.38,0.248724,0.435125,0.2565266715983058,0.004556,0.143525,0.46,0.28,"[Kd, F, q, Disp, C, alpha, L]","[[-0.5261415436673003, 0.5804747236052208]]","(-0.020736098363237032, 0.5181845809595654)",0.248724
8,8,"(Kd, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Di...",-0.738747,-0.731954,-0.6111834694960909,0.074398,-0.501248,0.34,0.3,-0.376761,-0.495855,-0.45420774975363354,-0.021335,-0.603071,0.38,0.18,"[F, q, Disp, C, alpha, Ks, L]","[[-3.905180055706234, -0.6562753667381214]]","(-0.6178182398930463, -0.135703814241236)",-0.376761
9,9,"(C, R-1, digraph{rho->Disp;rho->Ks;rho->Kd;Dis...",0.096289,0.099355,0.07422439759409916,0.005475,0.103954,0.40,0.48,0.090944,0.081929,0.06322807723405906,-0.002936,0.090625,0.43,0.47,"[Kd, q, Disp, alpha, Ks, L, rho, F]","[[-1.083784170431142, 1.339096203280483]]","(0.026587254889049228, 0.15530142069676267)",0.090944
